In [101]:
import json
from glob import glob
from tqdm import tqdm
import xml.etree.ElementTree as elemTree

In [108]:
def make_json():

    jsons = {}

    for data_type in ['Training', 'Validation']:
        temp_json = {
            'images':[],
            'categories':[],
            'annotations':[]
        }

        label_folders = sorted(glob(f'{data_type}/label/*'))
        
        #categories
        for _id, cat in enumerate(label_folders):
            cat = cat.split('/')[-1]
            temp_json['categories'].append(
                {
                    'supercategory':'Defect',
                    'id': _id,
                    'name':cat
                }
            )

        # images, annotations
        ann_id = 0
        for label_folder in tqdm(label_folders):
            label_files = sorted(glob(label_folder + '/*'))
            for label_file in label_files:
                if 'meta' in label_file:
                    f_name = label_file\
                        .replace('label', 'image')\
                        .replace('_meta.xml', '.jpg')
                    xml = open(label_file, 'rt', encoding='UTF8')
                    tree = elemTree.parse(xml)
                    root = tree.getroot()
                    item_no = root.find('div_cd').find('item_no').text
                    annotation = root.find('annotation')
                    size = annotation.find('size')
                    w = size.find('width').text
                    h = size.find('height').text
                    img_id = f_name.split('/')[-1].split('.')[0]
                    
                    temp_json['images'].append({
                        'file_name':f_name,
                        'height': int(h),
                        'width': int(w),
                        'id': img_id
                    })
                    for _object in annotation.findall('object'):
                        ann_id +=1
                        bbox = _object.find('bndbox')
                        xmin = float(bbox.find('xmin').text)
                        ymin = float(bbox.find('ymin').text)
                        xmax = float(bbox.find('xmax').text)
                        ymax = float(bbox.find('ymax').text)
                        ann_w = xmax-xmin
                        ann_h = ymax-ymin
                        temp_json['annotations'].append({
                            'id' : ann_id,
                            'image_id' : img_id,
                            'bbox':[xmin, ymin, ann_w, ann_h],
                            'area' : ann_w * ann_h,
                            'category_id':item_no,
                            'is_crowd' : 0,
                            'segmentation' : 0
                        })
        jsons[data_type] = temp_json
    
    return jsons

In [109]:
jsons = make_json()

100%|██████████| 205/205 [00:00<00:00, 483.28it/s]


In [118]:
import json

with open('train.json', mode='w') as f:
    json.dump(jsons['Training'], f)

In [119]:
with open('train.json', mode='r') as f:
    temp = json.load(f)

In [120]:
temp

{'images': [{'file_name': 'Training/image/10101_농심오징어짬뽕컵67G/10101_00_m_12.jpg',
   'height': 2988,
   'width': 2988,
   'id': '10101_00_m_12'},
  {'file_name': 'Training/image/10101_농심오징어짬뽕컵67G/10101_00_m_13.jpg',
   'height': 2988,
   'width': 2988,
   'id': '10101_00_m_13'},
  {'file_name': 'Training/image/10101_농심오징어짬뽕컵67G/10101_00_m_14.jpg',
   'height': 2988,
   'width': 2988,
   'id': '10101_00_m_14'},
  {'file_name': 'Training/image/10101_농심오징어짬뽕컵67G/10101_00_m_15.jpg',
   'height': 2988,
   'width': 2988,
   'id': '10101_00_m_15'},
  {'file_name': 'Training/image/10101_농심오징어짬뽕컵67G/10101_00_m_16.jpg',
   'height': 2988,
   'width': 2988,
   'id': '10101_00_m_16'},
  {'file_name': 'Training/image/10101_농심오징어짬뽕컵67G/10101_00_m_17.jpg',
   'height': 2988,
   'width': 2988,
   'id': '10101_00_m_17'},
  {'file_name': 'Training/image/10101_농심오징어짬뽕컵67G/10101_00_m_18.jpg',
   'height': 2988,
   'width': 2988,
   'id': '10101_00_m_18'},
  {'file_name': 'Training/image/10101_농심오징어짬뽕컵67G/10

# ------------------------------------------------------------------------------------

In [37]:
data_type='Training'

img_folders = sorted(glob(f'{data_type}/image/*'))
label_folders = sorted(glob(f'{data_type}/label/*'))

In [102]:
temp_json = {
    'images':[],
    'categories':[],
    'annotations':[]
}

#categories
for _id, cat in enumerate(label_folders):
    cat = cat.split('/')[-1]
    temp_json['categories'].append(
        {
            'supercategory':'Defect',
            'id': _id,
            'name':cat
        }
    )

# images, annotations
ann_id = 0
for label_folder in tqdm(label_folders):
    label_files = sorted(glob(label_folder + '/*'))
    for label_file in label_files:
        if 'meta' in label_file:
            f_name = label_file\
                .replace('label', 'image')\
                .replace('_meta.xml', '.jpg')
            xml = open(label_file, 'rt', encoding='UTF8')
            tree = elemTree.parse(xml)
            root = tree.getroot()
            item_no = root.find('div_cd').find('item_no').text
            annotation = root.find('annotation')
            size = annotation.find('size')
            w = size.find('width').text
            h = size.find('height').text
            img_id = f_name.split('/')[-1].split('.')[0]
            
            temp_json['images'].append({
                'file_name':f_name,
                'height': int(h),
                'width': int(w),
                'id': img_id
            })
            for _object in annotation.findall('object'):
                ann_id +=1
                bbox = _object.find('bndbox')
                xmin = float(bbox.find('xmin').text)
                ymin = float(bbox.find('ymin').text)
                xmax = float(bbox.find('xmax').text)
                ymax = float(bbox.find('ymax').text)
                ann_w = xmax-xmin
                ann_h = ymax-ymin
                temp_json['annotations'].append({
                    'id' : ann_id,
                    'image_id' : img_id,
                    'bbox':[xmin, ymin, ann_w, ann_h],
                    'area' : ann_w * ann_h,
                    'category_id':item_no,
                    'is_crowd' : 0,
                    'segmentation' : 0
                })
            # break
    
    # break

100%|██████████| 205/205 [00:03<00:00, 64.52it/s]


In [ ]:
from PIL import Image
# for folder_img in folder_imgs:
img = sorted(glob(folder_labels[0]+'/*'))[0]\
.replace('label','image').replace('.xml', '.jpg')
print(img)
Image.open(img)